In [3]:
# Use this initial code to work in the notebook as if it were a module, that 
# is, to be able to export classes and functions from other subpackages.

import pyprojroot
import sys

package_path = pyprojroot.here().__str__()
if package_path not in sys.path:
    sys.path.append(package_path)

In [4]:
from utils import AbsPaths
import os
import pandas as pd
import numpy as np
import glob
import re

In [5]:
def get_list_files(start_file_name = "atp_matches", path = None):
    if path is None:
        path = AbsPaths().get_abs_path_folder(folder_name="raw")

    return sorted(glob.glob(path + f"{start_file_name}" + "*.csv"))



In [6]:
def validate_year(list_files, year, index):
    if index not in [0,-1]:
        raise IndexError("You need provide 0 (start) or -1 (end)")
    if year is None:
        file = list_files[index]
        year = "/".join(re.findall(r"\d+",file.split(os.sep)[-1]))

    else:
        file = [f for f in list_files if year in f]

        if len(file) == 0:
            raise FileNotFoundError("You need provide an available start year")
        
        else:
            file = file[0]

    return file, year

In [7]:
def join_tables(start_year:str = None, start_file_name = "atp_matches", end_year = None, path_read = None, path_to_save = None):
    if path_read is None:
        path_read = AbsPaths().get_abs_path_folder(folder_name="raw")
    list_files = get_list_files(start_file_name = start_file_name, path = path_read)

    start_file, start_year = validate_year(list_files=list_files, year=start_year, index = 0)
    end_year = validate_year(list_files=list_files, year=end_year, index = -1)[1]

    cols = pd.read_csv(start_file,sep=",", nrows=1).columns
    data = pd.read_csv(start_file,sep=",", usecols=cols)

    for file in list_files:
        if file == start_file:
            pass
        year_file = "/".join(re.findall(r"\d+",file.split(os.sep)[-1]))
        # print(start_year, end_year, year_file)
        if (year_file > start_year) and (year_file <= end_year):

            cols_df = pd.read_csv(file,sep=",", nrows=1).columns
            df = pd.read_csv(file, sep=',', usecols=cols_df)
            data = pd.concat([data, df],ignore_index=True)
        
        data = data[data["tourney_level"] == "G"].reset_index(drop=True)
        
    if path_to_save is None:
        path_to_save = AbsPaths().get_abs_path_folder(folder_name="interim")
    
    data.to_csv(path_to_save + f"{start_file_name}_historic.csv", index=False)

    # return data

    

In [10]:
# join_tables(start_year="2010",end_year="2013")
join_tables()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced
0,2010-540,Wimbledon,Grass,128,G,20100621,1,103819,1.0,NaN,...,10.0,8.0,2.0,177.0,107.0,72.0,35.0,25.0,9.0,14.0
1,2010-540,Wimbledon,Grass,128,G,20100621,2,104592,NaN,Q,...,4.0,8.0,6.0,178.0,114.0,77.0,33.0,21.0,11.0,13.0
2,2010-540,Wimbledon,Grass,128,G,20100621,3,103096,NaN,NaN,...,4.0,18.0,5.0,125.0,75.0,53.0,26.0,20.0,5.0,8.0
3,2010-540,Wimbledon,Grass,128,G,20100621,4,103429,NaN,NaN,...,3.0,14.0,2.0,113.0,73.0,54.0,18.0,20.0,0.0,3.0
4,2010-540,Wimbledon,Grass,128,G,20100621,5,103852,22.0,NaN,...,12.0,6.0,4.0,134.0,77.0,46.0,33.0,19.0,12.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2027,2013-520,Roland Garros,Clay,128,G,20130527,123,103970,4.0,NaN,...,1.0,2.0,2.0,68.0,40.0,22.0,9.0,11.0,5.0,12.0
2028,2013-520,Roland Garros,Clay,128,G,20130527,124,104542,6.0,NaN,...,3.0,0.0,3.0,100.0,67.0,39.0,14.0,15.0,8.0,14.0
2029,2013-520,Roland Garros,Clay,128,G,20130527,125,104745,3.0,NaN,...,9.0,5.0,2.0,175.0,118.0,76.0,30.0,27.0,8.0,16.0
2030,2013-520,Roland Garros,Clay,128,G,20130527,126,103970,4.0,NaN,...,5.0,6.0,3.0,103.0,60.0,37.0,17.0,13.0,11.0,17.0
